In [1]:
print("....Starting chpater 4 ")

....Starting chpater 4 


In [2]:
GPT_CONFIG_124M = {
    "vocab_size":50257,
    "context_length":1024,
    "emb_dim":764,
    "n_heads":12,
    "n_layers":12,
    "drop_rate":0.1,
    "bias_":False
    }

In [25]:
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):
    def __init__(self, cfg): # ----->cfg stands for configuration of the model
        super().__init__()
        self.token_embedding=nn.Embedding(cfg['vocab_size'],cfg['emb_dim'])
        self.positional_embedding=nn.Embedding(cfg['context_length'],cfg['emb_dim'])
        self.drop_emnd=nn.Dropout(cfg['drop_rate'])
        self.transformer_blocks= nn.Sequential(
            *[DummyTransformerBlock(cfg)
              for _ in range(cfg['n_layers'])]
        )
        self.final_norm=DummyLayerNorm(cfg['emb_dim'])

        self.out_head=nn.Linear(
            cfg['emb_dim'],cfg['vocab_size']
            ,bias=False
        )


    def forward(self,in_idx):
        batch_size , seq_len= in_idx.shape

        tok_embeds=self.token_embedding(in_idx)
        pos_embeds=self.positional_embedding(
            torch.arange(seq_len,device=in_idx.device)
        ) 

        x = tok_embeds + pos_embeds
        x=self.drop_emnd(x)
        x=self.transformer_blocks(x)
        x=self.final_norm(x)

        logits=self.out_head(x)

        return logits
    

class DummyTransformerBlock(nn.Module):
    def __init__(self,cfg ):
        super().__init__()

    def forward(self,x):
        return x 

class DummyLayerNorm(nn.Module):
    def __init__(self,normalizerd_shape,eps=1e-5 ):
        super().__init__()

    def forward(self,x):
        return x                   



In [26]:
import tiktoken

GPT_toknizer=tiktoken.get_encoding('gpt2')
batch=[]
txt1="I regret pursuing "
txt2="I am an engineer"

batch.append(torch.tensor(GPT_toknizer.encode(txt1)))
batch.append(torch.tensor(GPT_toknizer.encode(txt2)))

batch=torch.stack(batch,dim=0)

batch

tensor([[   40, 13721, 15461,   220],
        [   40,   716,   281, 11949]])

In [27]:
torch.manual_seed(69)

model=DummyGPTModel(GPT_CONFIG_124M)

logits=model(batch)


logits.shape

torch.Size([2, 4, 50257])

In [28]:
logits

tensor([[[-0.1606,  0.1582, -0.1655,  ...,  0.2572,  0.0334, -0.6968],
         [-0.1840,  1.4080,  0.6589,  ..., -0.0227,  0.6340, -0.7125],
         [ 0.8344,  0.8508,  0.4825,  ...,  0.8057, -0.6171, -0.0570],
         [ 0.8624,  1.2597,  0.6285,  ...,  0.5382,  0.7182,  0.8064]],

        [[-0.0289,  0.0457,  0.4392,  ...,  0.2302,  0.1773, -1.3535],
         [ 0.3389,  0.4130, -0.8439,  ...,  0.6826,  0.7022, -1.4519],
         [ 0.6743,  1.1579,  0.9578,  ...,  1.3682, -0.9641, -0.5483],
         [-0.0676,  1.9398, -0.2413,  ...,  0.8942, -1.0402, -0.1722]]],
       grad_fn=<UnsafeViewBackward0>)

In [29]:
torch.manual_seed(69)

inputs=torch.rand(2,5)
demoNN=nn.Sequential(
    nn.Linear(5,6),
    nn.ReLU()
)
out=demoNN(inputs)
print(out)

tensor([[0.1514, 0.0000, 0.2192, 0.0000, 0.3302, 0.0000],
        [0.2511, 0.0000, 0.0000, 0.1048, 0.2901, 0.0000]],
       grad_fn=<ReluBackward0>)


In [36]:
out.sum(dim=-1,keepdim=True)/out.shape[-1]

tensor([[0.1168],
        [0.1077]], grad_fn=<DivBackward0>)

In [37]:
mean=out.mean(dim=-1,keepdim=True)
mean

tensor([[0.1168],
        [0.1077]], grad_fn=<MeanBackward1>)

In [46]:
variance=out.var(dim=-1,keepdim=True)
variance

tensor([[0.0196],
        [0.0177]], grad_fn=<VarBackward0>)

In [47]:
normalized_outputs = (out - mean)/torch.sqrt(variance)

In [51]:
normalized_mean=normalized_outputs.mean(dim=-1,keepdim=True)

In [53]:
normalized_variance=normalized_outputs.var(dim=-1,keepdim=True)


In [57]:
torch.set_printoptions(sci_mode=False)
print("mean:",normalized_mean)
print("variance:",normalized_variance)

mean: tensor([    0.0000,     0.0000], grad_fn=<MeanBackward1>)
variance: tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)


In [59]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps=1e-5
        self.scale=nn.Parameter(torch.ones(emb_dim))
        self.shift=nn.Parameter(torch.zeros(emb_dim))
    def forward(self,x):
        mean=x.mean(dim=-1,keep_dim=True)
        var=x.var(dim=-1,keep_dim=True , unbiased=False)
        norm_x=(x-mean) / torch.sqrt(var+self.eps)
        return self.scale * norm_x + self.shift

